In [ ]:

!apt-get update
!apt-get install -y wget unzip libvulkan1

!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f -y

!apt-get install -y xvfb

!pip install selenium chromedriver-autoinstaller

import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Ign:2 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [57.7 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,068 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security 

In [ ]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup


chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-images")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")

driver = webdriver.Chrome(options=chrome_options)
driver.set_page_load_timeout(30)
wait = WebDriverWait(driver, 15)


def login(driver):
    try:
        # have to click btn on paywall
        login_button = driver.find_element(By.CLASS_NAME, 'jp-paywall-subscribed')
        login_button.click()

        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'Username')))

        email_input = driver.find_element(By.ID, 'Username')
        email_input.send_keys('grp613@alumni.ku.dk')
        password_input = driver.find_element(By.NAME, 'Password')
        password_input.send_keys('xxxx')

        login_submit = driver.find_element(By.CSS_SELECTOR, 'button.ml-submit')
        login_submit.click()

        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'c-article-text-container')))
        print("Login successful.")
    except TimeoutException:
        print("Timeout while trying to login.")
    except NoSuchElementException:
        print("Unable to locate an element during login.")

def accept_cookies(driver):
    try:
        cookies_button = driver.find_element(By.ID, 'CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll')
        cookies_button.click()
    except NoSuchElementException:
        pass

def get_article_links():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    articles = soup.find_all('article')
    article_links = []
    for article in articles:
        a_tag = article.find('a')
        if a_tag:
            href = a_tag['href']
            full_url = "https://jyllands-posten.dk" + href
            article_links.append(full_url)
    return article_links

def scrape_article(article_url, term, retries=3):
    for attempt in range(retries):
        try:
            print(f"Scraping article: {article_url} (Attempt {attempt+1}/{retries})")
            driver.get(article_url)
            time.sleep(random.uniform(3, 7))

            accept_cookies(driver)

            try:
                paywall = driver.find_element(By.CLASS_NAME, 'jp-paywall-subscribed')
                if paywall:
                    print("Paywall detected. Logging in...")
                    login(driver)
            except NoSuchElementException:
                pass

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            try:
                title = soup.find('h1').get_text().strip()
            except:
                title = 'No title'

            try:
                subheader = soup.find('div', class_='c-article-top-info__description').get_text().strip()
            except:
                subheader = 'No subheader'

            try:
                date = soup.find('time')['datetime']
            except:
                date = 'No date'

            paragraphs = set()
            try:
                content_containers = soup.find_all('div', class_=[
                    'c-article-inline-element-container__text-width paywalled',
                    'c-article-text-container',
                    'c-article-inline-element-container__article-width',
                    'c-article-inline-element-container__full-width'
                ])

                # website changed, content in other containers than above:
                content_containers += soup.find_all('p', class_=['mb-3', 'mx-1', 'md:mx-auto', 'md:max-w-matrix-text'])

                for container in content_containers:
                    if container.name == 'p':
                        paragraphs.add(container.get_text().strip())
                    else:
                        for p in container.find_all('p'):
                            text = p.get_text().strip()
                            if text:
                                paragraphs.add(text)

                content = " ".join(paragraphs)

            except Exception as e:
                print(f"Error extracting content: {e}")
                content = 'No content'

            return {
                'url': article_url,
                'title': title,
                'subheader': subheader,
                'date': date,
                'content': content,
                'search_term': term
            }

        except (TimeoutException, NoSuchElementException) as e:
            print(f"Error scraping article {article_url}: {e}")
            time.sleep(5)

    return {'url': article_url, 'title': 'Failed to scrape', 'subheader': '', 'date': '', 'content': ''}

def load_all_articles(search_terms):
    all_articles = []

    for term in search_terms:
        page_number = 1
        while True:
            search_url = f"https://jyllands-posten.dk/soeg?term={term}&page={page_number}"
            print(f"Loading page: {search_url}")
            driver.get(search_url)
            time.sleep(random.uniform(3, 7))

            accept_cookies(driver)

            article_links = get_article_links()

            if not article_links:
                print("No more articles found.")
                break

            for article_url in article_links:
                article_data = scrape_article(article_url, term)
                all_articles.append(article_data)

            page_number += 1

    return all_articles


search_terms = ['ansigtsgenkendelse', 'facial recognition', 'face recognition', 'ansigtsaftryk', 'ansigtsscan',
                'masseovervågning ansigt', 'masseovervågning kamera', 'ansigtskamera', 'overvågningssamfund ansigt',
                'overvågningssamfund kamera', 'sikkerhedspakke overvågning', 'tryghedsskabende kamera']

scraped_articles = load_all_articles(search_terms)

df = pd.DataFrame(scraped_articles)
df.to_csv('jyllands_posten_scraped_articles.csv', index=False)
print(df.head())

driver.quit()


Loading page: https://jyllands-posten.dk/soeg?term=ansigtsgenkendelse&page=1
Scraping article: https://jyllands-posten.dk/indland/ECE17420099/folketinget-tillader-politiet-brug-af-ansigtsgenkendelse/ (Attempt 1/3)
Scraping article: https://jyllands-posten.dk/politik/ECE17403575/el-bakker-op-om-brug-af-ansigtsgenkendelse-i-sager-om-bandevold/ (Attempt 1/3)
Scraping article: https://jyllands-posten.dk/podcast/ECE17394059/hvad-skal-politiet-have-lov-at-bruge-ansigtsgenkendelse-til/ (Attempt 1/3)
Scraping article: https://jyllands-posten.dk/indland/ECE17397478/rigspolitiet-ser-stor-operativ-vaerdi-i-ansigtsgenkendelse-men-dommerforeningen-har-en-anke/ (Attempt 1/3)
Paywall detected. Logging in...
Login successful.
Scraping article: https://jyllands-posten.dk/indland/ECE17389092/ansigtsgenkendelse-er-paa-vej-i-danmark-central-direktoer-kommer-med-en-klar-advarsel/ (Attempt 1/3)
Scraping article: https://jyllands-posten.dk/indland/ECE17383307/dommere-vil-have-snor-i-politiets-brug-af-ansigts

In [ ]:
df

,url,title,subheader,date,content,search_term
0,https://jyllands-posten.dk/indland/ECE17420099...,Folketinget tillader politiet brug af ansigtsg...,No subheader,2024-10-24,"Aftalen er indgået onsdag af regeringen, SF, D...",ansigtsgenkendelse
1,https://jyllands-posten.dk/politik/ECE17403575...,EL bakker op om brug af ansigtsgenkendelse i s...,No subheader,2024-10-24,- Politiet er sat i verden for at opklare forb...,ansigtsgenkendelse
2,https://jyllands-posten.dk/podcast/ECE17394059...,Hvad skal politiet have lov at bruge ansigtsge...,No subheader,2024-08-29 04:30:00.000+0200,"Gæst: Thomas Jensen, direktør for Milestone Sy...",ansigtsgenkendelse
3,https://jyllands-posten.dk/indland/ECE17397478...,Rigspolitiet ser »stor operativ værdi« i ansig...,No subheader,2024-09-03 18:00:00.000+0200,»Jeg ser hverken domstole eller dommerkendelse...,ansigtsgenkendelse
4,https://jyllands-posten.dk/indland/ECE17389092...,Ansigtsgenkendelse er på vej i Danmark – centr...,No subheader,2024-09-01 08:00:00.000+0200,"Justitsminister Peter Hummelgaard anerkender, ...",ansigtsgenkendelse
...,...,...,...,...,...,...
440,https://jyllands-posten.dk/jpaarhus/ECE1374157...,"Østjyske kommuner oplyser gerne, hvor de overv...",No subheader,2022-02-17 06:45:00.000+0100,Forelagt indvendingerne konstaterede borgmeste...,tryghedsskabende kamera
441,https://jyllands-posten.dk/indland/ECE7636557/...,"500 øjne skal zoome ind, ud og gemme",No subheader,2015-04-18 23:00:00.000+0200,"I SR-regeringens terrorpakke, som blev lancere...",tryghedsskabende kamera
442,https://jyllands-posten.dk/indland/ECE7577506/...,Efter terrorangreb: Danskerne vil have mere v...,No subheader,2015-03-23 19:30:00.000+0100,"Han mener, at vi ikke længere ser overvågning ...",tryghedsskabende kamera
443,https://jyllands-posten.dk/erhverv/article6582...,Trygheds Gruppen åbner døren på klem,No subheader,2024-10-24,Men hvad så med at give pressen adgang til et ...,tryghedsskabende kamera


In [ ]:
df = df.drop_duplicates(subset=['url', 'date'], keep='first')
df

,url,title,subheader,date,content,search_term
0,https://jyllands-posten.dk/indland/ECE17420099...,Folketinget tillader politiet brug af ansigtsg...,No subheader,2024-10-24,"Aftalen er indgået onsdag af regeringen, SF, D...",ansigtsgenkendelse
1,https://jyllands-posten.dk/politik/ECE17403575...,EL bakker op om brug af ansigtsgenkendelse i s...,No subheader,2024-10-24,- Politiet er sat i verden for at opklare forb...,ansigtsgenkendelse
2,https://jyllands-posten.dk/podcast/ECE17394059...,Hvad skal politiet have lov at bruge ansigtsge...,No subheader,2024-08-29 04:30:00.000+0200,"Gæst: Thomas Jensen, direktør for Milestone Sy...",ansigtsgenkendelse
3,https://jyllands-posten.dk/indland/ECE17397478...,Rigspolitiet ser »stor operativ værdi« i ansig...,No subheader,2024-09-03 18:00:00.000+0200,»Jeg ser hverken domstole eller dommerkendelse...,ansigtsgenkendelse
4,https://jyllands-posten.dk/indland/ECE17389092...,Ansigtsgenkendelse er på vej i Danmark – centr...,No subheader,2024-09-01 08:00:00.000+0200,"Justitsminister Peter Hummelgaard anerkender, ...",ansigtsgenkendelse
...,...,...,...,...,...,...
440,https://jyllands-posten.dk/jpaarhus/ECE1374157...,"Østjyske kommuner oplyser gerne, hvor de overv...",No subheader,2022-02-17 06:45:00.000+0100,Forelagt indvendingerne konstaterede borgmeste...,tryghedsskabende kamera
441,https://jyllands-posten.dk/indland/ECE7636557/...,"500 øjne skal zoome ind, ud og gemme",No subheader,2015-04-18 23:00:00.000+0200,"I SR-regeringens terrorpakke, som blev lancere...",tryghedsskabende kamera
442,https://jyllands-posten.dk/indland/ECE7577506/...,Efter terrorangreb: Danskerne vil have mere v...,No subheader,2015-03-23 19:30:00.000+0100,"Han mener, at vi ikke længere ser overvågning ...",tryghedsskabende kamera
443,https://jyllands-posten.dk/erhverv/article6582...,Trygheds Gruppen åbner døren på klem,No subheader,2024-10-24,Men hvad så med at give pressen adgang til et ...,tryghedsskabende kamera


In [ ]:
df['date'] = df['date'].astype(str).str[:10]
df

<ipython-input-9-7c54668795f7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['date'].astype(str).str[:10]


,url,title,subheader,date,content,search_term
0,https://jyllands-posten.dk/indland/ECE17420099...,Folketinget tillader politiet brug af ansigtsg...,No subheader,2024-10-24,"Aftalen er indgået onsdag af regeringen, SF, D...",ansigtsgenkendelse
1,https://jyllands-posten.dk/politik/ECE17403575...,EL bakker op om brug af ansigtsgenkendelse i s...,No subheader,2024-10-24,- Politiet er sat i verden for at opklare forb...,ansigtsgenkendelse
2,https://jyllands-posten.dk/podcast/ECE17394059...,Hvad skal politiet have lov at bruge ansigtsge...,No subheader,2024-08-29,"Gæst: Thomas Jensen, direktør for Milestone Sy...",ansigtsgenkendelse
3,https://jyllands-posten.dk/indland/ECE17397478...,Rigspolitiet ser »stor operativ værdi« i ansig...,No subheader,2024-09-03,»Jeg ser hverken domstole eller dommerkendelse...,ansigtsgenkendelse
4,https://jyllands-posten.dk/indland/ECE17389092...,Ansigtsgenkendelse er på vej i Danmark – centr...,No subheader,2024-09-01,"Justitsminister Peter Hummelgaard anerkender, ...",ansigtsgenkendelse
...,...,...,...,...,...,...
440,https://jyllands-posten.dk/jpaarhus/ECE1374157...,"Østjyske kommuner oplyser gerne, hvor de overv...",No subheader,2022-02-17,Forelagt indvendingerne konstaterede borgmeste...,tryghedsskabende kamera
441,https://jyllands-posten.dk/indland/ECE7636557/...,"500 øjne skal zoome ind, ud og gemme",No subheader,2015-04-18,"I SR-regeringens terrorpakke, som blev lancere...",tryghedsskabende kamera
442,https://jyllands-posten.dk/indland/ECE7577506/...,Efter terrorangreb: Danskerne vil have mere v...,No subheader,2015-03-23,"Han mener, at vi ikke længere ser overvågning ...",tryghedsskabende kamera
443,https://jyllands-posten.dk/erhverv/article6582...,Trygheds Gruppen åbner døren på klem,No subheader,2024-10-24,Men hvad så med at give pressen adgang til et ...,tryghedsskabende kamera


In [ ]:
df['merged_content'] = df['title'] + ' ' + df['content']

<ipython-input-11-f0ae02cf3714>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['merged_content'] = df['title'] + ' ' + df['content']


In [ ]:
df['merged_content'][20]

'Rigspolitichef frabeder sig grænsekontrol af lovlydige borgere - Politiet vil gerne kontrollere miljøerne, herunder når de færdes på grænsen. Men det karakteristiske for bestillingskriminalitet er ifølge politichefen, at de kriminelle ikke har våben med over grænsen. - Det handler om, hvad vi kigger efter. Vi kigger efter ting, som er baseret på de efterretninger, vi har. Vi kigger ikke tilfældigt. - Vi har interesse i at overvåge nogle bestemte personer, nogle bestemte køretøjer, nogle bestemte netværk. Når de krydser grænsen, så holder vi øje med det, siger Thorkild Fogde. Politiet har ingen interesse i en omfattende grænsekontrol af borgere, som krydser grænsen mellem Sverige og Danmark. Det er ifølge rigspolitichefen uinteressant for politiet at kunne overvåge grænsetrafik generelt. Samtidig styrker man samarbejdet med svenske myndigheder. Flere partier har krævet grænsekontrol, herunder Danmarksdemokraterne, som ønsker en grænsekontrol i lighed med den, der findes på grænsen til 

In [ ]:
from google.colab import files
df.to_csv('JP.csv', index=False)
files.download('JP.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>